In [1]:
import sys
sys.path.append('../')

from handles.marketdata import *
# import json
# from instruments.swap import *
# from datetime import datetime, timedelta
# import pandas as pd
# import matplotlib.pyplot as plt
# import tensorflow as tf
# from utilities.targetcalendar import TARGET
# import re
# from datetime import datetime, timedelta

# Market Data

In [3]:
mdl = MarketDataLoader()

In [10]:
# mdl.ir_eur_discount_1m
# mdl.ir_eur_discount_3m
mdl.ir_eur_discount_6m
# mdl.ir_eur_discount_12m
mdl.ir_eur_discount_estr

,type,name,curve_instrument,quote_type,market_quote,maturity_date,daycount
0,IR,EUR-ESTR-ON,DF-29-Sep-2023,MID,1.000000,2023-09-29,0
1,IR,EUR-ESTR-ON,DF-02-Oct-2023,MID,0.999675,2023-10-02,3
2,IR,EUR-ESTR-ON,DF-03-Oct-2023,MID,0.999566,2023-10-03,4
3,IR,EUR-ESTR-ON,DF-04-Oct-2023,MID,0.999458,2023-10-04,5
4,IR,EUR-ESTR-ON,DF-10-Oct-2023,MID,0.998808,2023-10-10,11
...,...,...,...,...,...,...,...
58,IR,EUR-ESTR-ON,DF-03-Oct-2053,MID,0.411540,2053-10-03,10962
59,IR,EUR-ESTR-ON,DF-03-Oct-2058,MID,0.367475,2058-10-03,12788
60,IR,EUR-ESTR-ON,DF-03-Oct-2063,MID,0.328289,2063-10-03,14614
61,IR,EUR-ESTR-ON,DF-03-Oct-2073,MID,0.267781,2073-10-03,18267


# Curve Handle

# layer esempio

In [2]:
class Market:
    def __init__(self) -> None:
        self.market_dfs = pd.read_csv('../data/markets/Market_2023_09_29_DFs.csv', 
                                        skiprows= 2,
                                        on_bad_lines='skip')
        self.evaluation_date = date(2023,9,29)
        Settings.evaluation_date = self.evaluation_date
        self._eur_6m_index_init()
    
    @property
    def estr_curve(self):
        df_eur = self.market_dfs[self.market_dfs['Property'].str.contains("IR.EUR-ESTR-ON.DF")][['Property', 'Label']]
        dates = []
        for i in range(len(df_eur['Property'].values)):
            str_temp = df_eur['Property'].values[i][18:29]
            dates.append(datetime.strptime(str_temp, '%d-%b-%Y') )
        dfs_pillars = [(dates[t]-dates[0]).days/365 for t in range(len(dates))]
        dfs_market = df_eur['Label'].values
        return DiscountCurveSimple(dfs_pillars, dfs_market)
    
    def _eur_6m_index_init(self):
        index_eur_6m = IborIndex("EUR6M", "calendar", "6M")
        # fixing storici settati costanti a 3% 
        dates_random = [date(2023,9,28) - timedelta(x) for x in range(0,200)] 
        [index_eur_6m.add_fixing(dt, 0.03) for dt in dates_random]
        self._eur_6m_index = index_eur_6m 
    
    @property
    def eur_6m_index(self):
        return self._eur_6m_index
    
## market instance
md = Market()

In [3]:
def make_swap(trade):
    ''' 
    Read json and returns tq.Swap
    '''
    leg1_schedule_len = len(trade['data']['Leg1.schedule']['rows'])
    leg2_schedule_len = len(trade['data']['Leg2.schedule']['rows'])

    leg1_is_fixed = trade['data']['Leg1.isFixedLeg']
    leg2_is_fixed = trade['data']['Leg2.isFixedLeg']

    if not leg1_is_fixed:
        leg1_index = trade['data']['Leg1.index']
    elif not leg2_is_fixed:
        leg2_index = trade['data']['Leg2.index']

    date_format = '%Y-%m-%d'
    leg1_payment_dates = [trade['data']['Leg1.schedule']['rows'][i]['paymentDate'] for i in range(leg1_schedule_len)]
    leg1_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg1_payment_dates]
    leg2_payment_dates = [trade['data']['Leg2.schedule']['rows'][i]['paymentDate'] for i in range(leg2_schedule_len)]
    leg2_payment_dates = [datetime.strptime(dt, date_format).date() for dt in leg2_payment_dates]

    leg1_notionals = [trade['data']['Leg1.notionalTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_notionals = [trade['data']['Leg2.notionalTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_margins = [trade['data']['Leg1.marginTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_margins = [trade['data']['Leg2.marginTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_rate_factor = [trade['data']['Leg1.rateFactorTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_rate_factor = [trade['data']['Leg2.rateFactorTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    leg1_fixed_payments = [trade['data']['Leg1.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg1_schedule_len)]
    leg2_fixed_payments = [trade['data']['Leg2.fixedPaymentAmountTable']['rows'][i]['value'] for i in range(leg2_schedule_len)]

    if leg1_is_fixed and not leg2_is_fixed:

        if leg2_index == 'EUR-EURIBOR-6M':
            index = md.eur_6m_index
        else:
            raise ValueError('No index available')

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals ,
                                    leg2_rate_factor, # moltiplicatore variabile
                                    leg2_margins, # spread variabile
                                    index,
                                    leg1_margins) # tasso fisso
        
    elif leg2_is_fixed and not leg1_is_fixed:
        if leg1_index == 'EUR-EURIBOR-6M':
            index = md.eur_6m_index
        else:
            raise ValueError('No index available')

        swap_obj = SwapFixedFloating(leg1_payment_dates,
                                    leg2_payment_dates,
                                    leg1_notionals,
                                    leg2_notionals,
                                    leg1_rate_factor, # moltiplicatore variabile
                                    leg1_margins, # spread variabile
                                    index,
                                    leg2_margins) # tasso fisso
    
    return swap_obj

def make_portfolio(trade_data):
    portfolio = []
    errors = []
    for trade_json in trade_data:
        try:
            if trade_json['data']['productType'] == 'InterestRateSwapScript':
                trade_json['trade_object'] = make_swap(trade_json)
                portfolio.append(trade_json)
        except:
            errors.append(trade_json['ref'])
    return portfolio, errors


In [4]:
with open('../data/trades/TradesAggregated_2.json', 'r') as file:
    trade_data = json.load(file)

ptf, mapping_errors = make_portfolio(trade_data)

#filtro per swap plain 
for trade_json in ptf:
    if trade_json['ref'] == '0-17092454':
        trade = trade_json
        
swap_obj = trade['trade_object']

# esempio curva dummy

In [6]:
import math
import tensorflow as tf
from tensorflow.python.framework import dtypes

class RateCurve:
    def __init__(self, pillars, rates):
        self.pillars = pillars # list
        self.rates = rates     # tensor list

    def discount(self, term):
        if term <= self.pillars[0]:
            nrt = -term * self.rates[0]
            df = tf.exp(nrt)
            return df
        if term >= self.pillars[-1]:
            nrt = -term * self.rates[-1]
            df = tf.exp(nrt)
            return df
        for i in range(0, len(self.pillars) - 1):
            if term < self.pillars[i + 1]:
                dtr = 1 / (self.pillars[i + 1] - self.pillars[i])
                w1 = (self.pillars[i + 1] - term) * dtr
                w2 = (term - self.pillars[i]) * dtr
                r1 = w1 * self.rates[i]
                r2 = w2 * self.rates[i + 1]
                rm = r1 + r2
                nrt = -term * rm
                df = tf.exp(nrt)
                return df


In [7]:
rc_pillars = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0]
rc_rates = [0.052, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]

rc_rates_tensor = [tf.Variable(r, dtype=dtypes.float64) for r in rc_rates]
curve_tensor = RateCurve(rc_pillars, rc_rates_tensor)

In [8]:
with tf.GradientTape() as tape:
    npv = swap_obj.price(curve_tensor, curve_tensor, md.evaluation_date)
    #sum = tf.reduce_sum(npv, 0)

In [9]:
sensitivities = tape.gradient(npv, curve_tensor.rates)

In [10]:
sensitivities

[<tf.Tensor: shape=(), dtype=float64, numpy=169578.59327819652>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-170924.58151411882>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-837887.8452326212>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-4603783.867229899>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-15279632.598827647>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-26670314.8960332>,
 <tf.Tensor: shape=(), dtype=float64, numpy=-6343871.905771269>,
 None]